# Kalman Filter

## Imports

In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from tqdm import tqdm

## Load all csv names

In [6]:
f = open("./data/data_names.txt", "r")
file_names = map(lambda x: x[:-1], f.readlines())
f.close()

csv_names = list(file_names)
csv_names

68

## Plot functions

In [4]:
variables_dict = {'speed': 'Velocidad', 'latitude':'Lat', 'longitude':'lng', 'accPosition': 'Presión del acelerador',
                  'accX': 'Aceleración en X', 'accY': 'Aceleración en Y', 'accZ': 'Aceleración en Z',
                  'magX': 'Fuerza magnética en X', 'magY': 'Fuerza magnética en Y', 'magZ': 'Fuerza magnética en Z',
                  'velAngX': 'Velocidad angular en X', 'velAngY': 'Velocidad angular en Y', 'velAngZ': 'Velocidad angular en Z'}
units_dict = {'speed': 'm/s', 'latitude':'°', 'longitude':'°', 'accPosition': '% de presión',
              'accX': 'm/s\u00B2', 'accY': 'm/s\u00B2', 'accZ': 'm/s\u00B2',
              'magX': '\u03BC T', 'magY': '\u03BC T', 'magZ': '\u03BC T',
              'velAngX': 'rad/seg', 'velAngY': 'rad/seg', 'velAngZ': 'rad/seg'}


def update_layout(fig, chart_title: str, **kwargs):
    """Update layout for every chart

    Args:
        fig: A plotly figure
        chart_title (str): A title for chart

    Returns:
        fig: the updated layout for the plotly figure
    """
    # Get kwards data if is needed
    x_title = kwargs.get('xaxis_title', None)
    y_title = kwargs.get('yaxis_title', None)
    height_size = kwargs.get('height', 450)

    # Update layout of the figure
    fig.update_layout(
        title={'text': chart_title, 'x': 0.5},
        legend_title='Tipo de evento',
        xaxis_title=x_title,
        yaxis_title=y_title,
        template='plotly_white',
        autosize=True,
        height=height_size,
        font=dict(
            family="BlinkMacSystemFont,-apple-system,Segoe UI,Roboto,Oxygen,Ubuntu, \
                    Cantarell,Fira Sans,Droid Sans,Helvetica Neue,Helvetica,Arial,sans-serif",
            size=14,
            color="#363636"
        )
    )

    return fig


def line_chart(df, Y_measured ,Y_hat):
    """Line chart created with plotly graphic object

    Args:
        df (DataFrame): The dataframe that contains the data to plot
        variable (str): The specific variable to plot

    Returns:
        fig: A plotly figure
    """
    x = df['timestamp']
    variable = Y_measured.name
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=Y_measured,
        name='Measured data',
        marker_color='#00D1B1',
        text=df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))
    fig.add_trace(go.Scatter(
        x=x,
        y=Y_hat,
        name='Filter data',
        marker_color='#3273dc',
        text=df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))

    fig = update_layout(fig, f'Gráfico de linea de la {variables_dict[variable]}',
                        xaxis_title='Marca de Tiempo', yaxis_title=units_dict[variable])
    fig.update_yaxes(nticks=12)
    fig.update_xaxes(nticks=12, tickangle=45)

    return fig

## Using filter

In [5]:
from pykalman import KalmanFilter

variables = ["accX", "accY", "accZ", "velAngX", "velAngY", "velAngZ", "magX", "magY", "magZ"]
for name in csv_names:
    df = pd.read_csv(f"./data/{name}")
    for var in tqdm(variables, desc=name):
        data = df[var]

        kf = KalmanFilter(initial_state_mean = data.iloc[0], n_dim_obs=1)
        filter_data = kf.em(data).filter(data)[0].T[0]
        filter_data_s = pd.Series(np.array(filter_data), name=var)
        df[var] = filter_data_s

        #fig = line_chart(df, data, new_data_s)
        #fig.show()

    df.to_csv(f"./filtered_data/{name}", index=False)

smartphone_02-Feb-2022-20-14_giro izquierdo agresivo_Data-Mv2JuPQFlZdPc9e8f1U.csv: 100%|██████████| 9/9 [00:15<00:00,  1.73s/it]
smartphone_02-Feb-2022-20-14_giro izquierdo agresivo_Data-Mv2JXMh1X_V30cbr0Mw.csv: 100%|██████████| 9/9 [00:29<00:00,  3.27s/it]
smartphone_02-Feb-2022-20-15_giro izquierdo agresivo_Data-Mv2KZyC2nSbvYeWyEmh.csv: 100%|██████████| 9/9 [00:39<00:00,  4.41s/it]
smartphone_02-Feb-2022-20-19_cambio de línea agresivo izquierda_Data-Mv2L_EBJR_gDU3LpwVs.csv: 100%|██████████| 9/9 [00:21<00:00,  2.42s/it]
smartphone_02-Feb-2022-20-21_cambio de línea agresivo izquierda_Data-Mv2LixfwEokL4QCgN_p.csv: 100%|██████████| 9/9 [00:32<00:00,  3.57s/it]
smartphone_02-Feb-2022-20-22_cambio de línea agresivo izquierda_Data-Mv2MN6xv2W3Rv1Rmb_Z.csv: 100%|██████████| 9/9 [00:26<00:00,  2.97s/it]
smartphone_02-Feb-2022-20-23_cambio de línea agresivo izquierda_Data-Mv2MemmvWzydRwVTgDS.csv: 100%|██████████| 9/9 [00:20<00:00,  2.25s/it]
smartphone_02-Feb-2022-20-26_cambio de línea agresivo

## Plot filter data

In [9]:
df = pd.read_csv("./data/smartphone_02-Feb-2022-20-31_giro derecho agresivo_Data-Mv2Nk_4aV4M_KufahA0.csv")
df_filter = pd.read_csv("./filtered_data/smartphone_02-Feb-2022-20-31_giro derecho agresivo_Data-Mv2Nk_4aV4M_KufahA0.csv")
for var in variables:
    data = df[var]
    filter_data = df_filter[var]
    fig = line_chart(df, data, filter_data)
    fig.show()

In [90]:
from sklearn.metrics import mean_squared_error

max_standby = 7250
min_standby = 6680
var_noise = "accZ"


g_noise = df.loc[(df['id'] <= max_standby) & (df["id"] >=min_standby), var_noise]
g_noise

y_true = np.array([10]*len(g_noise))

mean_squared_error(y_true, g_noise)

1.2758263365434335